# Session 5 – Orchestrateur Multi-Agent

Démontre un pipeline simple à deux agents (Chercheur -> Éditeur) en utilisant Foundry Local.


### Explication : Installation des dépendances
Installe `foundry-local-sdk` et `openai` nécessaires pour l'accès aux modèles locaux et les complétions de chat. Idempotent.


# Scénario
Implémente un modèle minimal d'orchestrateur à deux agents :
- **Agent chercheur** recueille des informations factuelles concises
- **Agent éditeur** reformule pour une clarté exécutive

Démontre une mémoire partagée par agent, un passage séquentiel des résultats intermédiaires et une fonction de pipeline simple. Extensible à davantage de rôles (par exemple, Critique, Vérificateur) ou à des branches parallèles.

**Variables d'environnement :**
- `FOUNDRY_LOCAL_ALIAS` - Modèle par défaut à utiliser (par défaut : phi-4-mini)
- `AGENT_MODEL_PRIMARY` - Modèle principal de l'agent (remplace ALIAS)
- `AGENT_MODEL_EDITOR` - Modèle de l'agent éditeur (par défaut : modèle principal)

**Référence SDK :** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local

**Comment ça fonctionne :**
1. **FoundryLocalManager** démarre automatiquement le service Foundry Local
2. Télécharge et charge le modèle spécifié (ou utilise la version mise en cache)
3. Fournit un point de terminaison compatible avec OpenAI pour l'interaction
4. Chaque agent peut utiliser un modèle différent pour des tâches spécialisées
5. Une logique de reprise intégrée gère les échecs transitoires de manière fluide

**Caractéristiques principales :**
- ✅ Découverte et initialisation automatique du service
- ✅ Gestion du cycle de vie des modèles (téléchargement, mise en cache, chargement)
- ✅ Compatibilité avec le SDK OpenAI pour une API familière
- ✅ Support multi-modèles pour la spécialisation des agents
- ✅ Gestion robuste des erreurs avec logique de reprise
- ✅ Inférence locale (aucune API cloud requise)


In [16]:
# Install dependencies
!pip install -q foundry-local-sdk openai

### Explication : Importations principales et typage
Introduction des dataclasses pour le stockage des messages des agents et des indications de typage pour plus de clarté. Importation du gestionnaire local Foundry + client OpenAI pour les actions ultérieures des agents.


In [17]:
from dataclasses import dataclass, field
from typing import List
import os
from foundry_local import FoundryLocalManager
from openai import OpenAI

### Explication : Initialisation du modèle (Modèle SDK)
Utilise le SDK Python Foundry Local pour une gestion robuste des modèles :
- **FoundryLocalManager(alias)** - Démarre automatiquement le service et charge le modèle via un alias
- **get_model_info(alias)** - Résout l'alias en identifiant concret du modèle
- **manager.endpoint** - Fournit le point de terminaison du service pour le client OpenAI
- **manager.api_key** - Fournit la clé API (optionnelle pour une utilisation locale)
- Prend en charge des modèles distincts pour différents agents (principal vs éditeur)
- Logique de reprise intégrée avec un backoff exponentiel pour plus de résilience
- Vérification de la connexion pour garantir que le service est prêt

**Modèle clé du SDK :**
```python
manager = FoundryLocalManager(alias)
model_info = manager.get_model_info(alias)
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key)
```

**Gestion du cycle de vie :**
- Les gestionnaires sont stockés globalement pour un nettoyage approprié
- Chaque agent peut utiliser un modèle différent pour se spécialiser
- Découverte automatique des services et gestion des connexions
- Reprise en douceur avec un backoff exponentiel en cas d'échec

Cela garantit une initialisation correcte avant le début de l'orchestration des agents.

**Référence :** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local


In [18]:
import time

# Environment configuration
PRIMARY_ALIAS = os.getenv('AGENT_MODEL_PRIMARY', os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini'))
EDITOR_ALIAS = os.getenv('AGENT_MODEL_EDITOR', PRIMARY_ALIAS)

# Store managers globally for proper lifecycle management
primary_manager = None
editor_manager = None

def init_model(alias: str, max_retries: int = 3):
    """Initialize Foundry Local manager with retry logic.
    
    Args:
        alias: Model alias to initialize
        max_retries: Number of retry attempts with exponential backoff
    
    Returns:
        Tuple of (manager, client, model_id, endpoint)
    """
    delay = 2.0
    last_err = None
    
    for attempt in range(1, max_retries + 1):
        try:
            print(f"[Init] Starting Foundry Local for '{alias}' (attempt {attempt}/{max_retries})...")
            
            # Initialize manager - this starts the service and loads the model
            manager = FoundryLocalManager(alias)
            
            # Get model info to retrieve the actual model ID
            model_info = manager.get_model_info(alias)
            model_id = model_info.id
            
            # Create OpenAI client with manager's endpoint
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key or 'not-needed'
            )
            
            # Verify the connection with a simple test
            models = client.models.list()
            print(f"[OK] Initialized '{alias}' -> {model_id} at {manager.endpoint}")
            
            return manager, client, model_id, manager.endpoint
            
        except Exception as e:
            last_err = e
            if attempt < max_retries:
                print(f"[Retry {attempt}/{max_retries}] Failed to init '{alias}': {e}")
                print(f"[Retry] Waiting {delay:.1f}s before retry...")
                time.sleep(delay)
                delay *= 2
            else:
                print(f"[ERROR] Failed to initialize '{alias}' after {max_retries} attempts")
    
    raise RuntimeError(f"Failed to initialize '{alias}' after {max_retries} attempts: {last_err}")

# Initialize primary model (for researcher)
print(f"\n{'='*80}")
print(f"Initializing Primary Model: {PRIMARY_ALIAS}")
print('='*80)
primary_manager, primary_client, PRIMARY_MODEL_ID, primary_endpoint = init_model(PRIMARY_ALIAS)

# Initialize editor model (may be same as primary)
if EDITOR_ALIAS != PRIMARY_ALIAS:
    print(f"\n{'='*80}")
    print(f"Initializing Editor Model: {EDITOR_ALIAS}")
    print('='*80)
    editor_manager, editor_client, EDITOR_MODEL_ID, editor_endpoint = init_model(EDITOR_ALIAS)
else:
    print(f"\n[Info] Editor using same model as primary")
    editor_manager = primary_manager
    editor_client, EDITOR_MODEL_ID = primary_client, PRIMARY_MODEL_ID
    editor_endpoint = primary_endpoint

print(f"\n{'='*80}")
print(f"[Configuration Summary]")
print('='*80)
print(f"  Primary Agent:")
print(f"    - Alias: {PRIMARY_ALIAS}")
print(f"    - Model: {PRIMARY_MODEL_ID}")
print(f"    - Endpoint: {primary_endpoint}")
print(f"\n  Editor Agent:")
print(f"    - Alias: {EDITOR_ALIAS}")
print(f"    - Model: {EDITOR_MODEL_ID}")
print(f"    - Endpoint: {editor_endpoint}")
print('='*80)



Initializing Primary Model: phi-4-mini
[Init] Starting Foundry Local for 'phi-4-mini' (attempt 1/3)...
[OK] Initialized 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4 at http://127.0.0.1:59959/v1

Initializing Editor Model: gpt-oss-20b
[Init] Starting Foundry Local for 'gpt-oss-20b' (attempt 1/3)...
[OK] Initialized 'gpt-oss-20b' -> gpt-oss-20b-cuda-gpu:1 at http://127.0.0.1:59959/v1

[Configuration Summary]
  Primary Agent:
    - Alias: phi-4-mini
    - Model: Phi-4-mini-instruct-cuda-gpu:4
    - Endpoint: http://127.0.0.1:59959/v1

  Editor Agent:
    - Alias: gpt-oss-20b
    - Model: gpt-oss-20b-cuda-gpu:1
    - Endpoint: http://127.0.0.1:59959/v1


### Explication : Classes Agent & Memory
Définit un `AgentMsg` léger pour les entrées de mémoire et un `Agent` qui encapsule :
- **Rôle système** - La personnalité et les instructions de l'agent
- **Historique des messages** - Conserve le contexte de la conversation
- **Méthode act()** - Exécute des actions avec une gestion appropriée des erreurs

L'agent peut utiliser différents modèles (principal vs éditeur) et maintient un contexte isolé par agent. Ce modèle permet :
- La persistance de la mémoire entre les actions
- Une attribution flexible des modèles par agent
- L'isolation et la récupération des erreurs
- Une orchestration et un chaînage simplifiés


In [19]:
@dataclass
class AgentMsg:
    role: str
    content: str

@dataclass
class Agent:
    name: str
    system: str
    client: OpenAI = None  # Allow per-agent client assignment
    model_id: str = None   # Allow per-agent model
    memory: List[AgentMsg] = field(default_factory=list)

    def _history(self):
        """Return chat history in OpenAI messages format including system + memory."""
        msgs = [{'role': 'system', 'content': self.system}]
        for m in self.memory[-6:]:  # Keep last 6 messages to avoid context overflow
            msgs.append({'role': m.role, 'content': m.content})
        return msgs

    def act(self, prompt: str, temperature: float = 0.4, max_tokens: int = 300):
        """Send a prompt, store user + assistant messages in memory, and return assistant text.
        
        Args:
            prompt: User input/task for the agent
            temperature: Sampling temperature (0.0-1.0)
            max_tokens: Maximum tokens to generate
        
        Returns:
            Assistant response text
        """
        # Use agent-specific client/model or fall back to primary
        client_to_use = self.client or primary_client
        model_to_use = self.model_id or PRIMARY_MODEL_ID
        
        self.memory.append(AgentMsg('user', prompt))
        
        try:
            # Build messages including system prompt and history
            messages = self._history() + [{'role': 'user', 'content': prompt}]
            
            resp = client_to_use.chat.completions.create(
                model=model_to_use,
                messages=messages,
                max_tokens=max_tokens,
                temperature=temperature,
            )
            
            # Validate response
            if not resp.choices:
                raise RuntimeError("No completion choices returned")
            
            out = resp.choices[0].message.content or ""
            
            if not out:
                raise RuntimeError("Empty response content")
            
        except Exception as e:
            out = f"[ERROR:{self.name}] {type(e).__name__}: {str(e)}"
            print(f"[Agent Error] {self.name}: {type(e).__name__}: {str(e)}")
        
        self.memory.append(AgentMsg('assistant', out))
        return out

print("[INFO] Agent classes initialized with Foundry SDK support")
print(f"[INFO] Using OpenAI SDK version: {OpenAI.__module__}")


[INFO] Agent classes initialized with Foundry SDK support
[INFO] Using OpenAI SDK version: openai


### Explication : Pipeline orchestré
Crée deux agents spécialisés :
- **Chercheur** : Utilise le modèle principal, collecte des informations factuelles
- **Éditeur** : Peut utiliser un modèle distinct (si configuré), affine et réécrit

La fonction `pipeline` :
1. Le chercheur collecte des informations brutes
2. L'éditeur les affine pour produire un contenu prêt à être présenté
3. Retourne à la fois les résultats intermédiaires et finaux

Ce modèle permet :
- La spécialisation des modèles (différents modèles pour différents rôles)
- L'amélioration de la qualité grâce à un traitement en plusieurs étapes
- La traçabilité de la transformation des informations
- Une extension facile à davantage d'agents ou à un traitement parallèle


In [ ]:
# Create specialized agents with optional model assignment
researcher = Agent(
    name='Researcher',
    system='You collect concise factual bullet points.',
    client=primary_client,
    model_id=PRIMARY_MODEL_ID
)

editor = Agent(
    name='Editor',
    system='You rewrite content for clarity and an executive, action-focused tone.',
    client=editor_client,
    model_id=EDITOR_MODEL_ID
)

def pipeline(q: str, verbose: bool = True):
    """Execute multi-agent pipeline: Researcher -> Editor.
    
    Args:
        q: User question/task
        verbose: Print intermediate outputs
    
    Returns:
        Dictionary with research, final outputs, and metadata
    """
    if verbose:
        print(f"[Pipeline] Question: {q}\n")
    
    # Stage 1: Research
    if verbose:
        print("[Stage 1: Research]")
    research = researcher.act(q)
    if verbose:
        print(f"Output: {research[:200]}...\n")
    
    # Stage 2: Editorial refinement
    if verbose:
        print("[Stage 2: Editorial Refinement]")
    rewrite = editor.act(
        f"Rewrite professionally with a 1-sentence executive summary first. "
        f"Improve clarity, keep bullet structure if present. Source:\n{research}"
    )
    if verbose:
        print(f"Output: {rewrite[:200]}...\n")
    
    return {
        'question': q,
        'research': research,
        'final': rewrite,
        'models': {
            'researcher': PRIMARY_MODEL_ID,
            'editor': EDITOR_MODEL_ID
        }
    }

# Execute sample pipeline
print("="*80)
result = pipeline('Explain why edge AI matters for compliance and latency.')
print("="*80)
print("\n[FINAL OUTPUT]")
print(result['final'])
print("\n[METADATA]")
print(f"Models used: {result['models']}")
result

[Pipeline] Question: Explain why edge AI matters for compliance and latency.

[Stage 1: Research]
Output: - **Data Sovereignty**: Edge AI allows data to be processed locally, which can help organizations comply with regional data protection regulations by keeping sensitive information within the borders o...

[Stage 2: Editorial Refinement]


### Explication : Exécution du pipeline et résultats
Exécute le pipeline multi-agent sur une question axée sur la conformité et la latence pour démontrer :
- La transformation de l'information en plusieurs étapes
- La spécialisation et la collaboration des agents
- L'amélioration de la qualité des résultats grâce au raffinement
- La traçabilité (les sorties intermédiaires et finales sont conservées)

**Structure des résultats :**
- `question` - Requête originale de l'utilisateur
- `research` - Résultat brut de la recherche (points factuels)
- `final` - Résumé exécutif affiné
- `models` - Modèles utilisés à chaque étape

**Idées d'extension :**
1. Ajouter un agent Critique pour la révision de la qualité
2. Mettre en œuvre des agents de recherche parallèles pour différents aspects
3. Ajouter un agent Vérificateur pour la vérification des faits
4. Utiliser différents modèles selon les niveaux de complexité
5. Implémenter des boucles de rétroaction pour une amélioration itérative


### Avancé : Configuration personnalisée de l'agent

Essayez de personnaliser le comportement de l'agent en modifiant les variables d'environnement avant d'exécuter la cellule d'initialisation :

**Modèles disponibles :**
- Utilisez `foundry model ls` dans le terminal pour voir tous les modèles disponibles
- Exemples : phi-4-mini, phi-3.5-mini, qwen2.5-7b, llama-3.2-3b, etc.


In [ ]:
# Example: Use different models for different agents
# Uncomment and modify as needed:

# import os
# os.environ['AGENT_MODEL_PRIMARY'] = 'phi-4-mini'      # Fast, good for research
# os.environ['AGENT_MODEL_EDITOR'] = 'qwen2.5-7b'       # Higher quality for editing

# Then restart the kernel and re-run all cells

# Test with different questions
test_questions = [
    "What are 3 key benefits of using small language models?",
    "How does RAG improve AI accuracy?",
    "Why is local inference important for privacy?"
]

print("Testing pipeline with multiple questions:\n")
for i, q in enumerate(test_questions, 1):
    print(f"\n{'='*80}")
    print(f"Question {i}: {q}")
    print('='*80)
    r = pipeline(q, verbose=False)
    print(f"\n[FINAL]: {r['final'][:300]}...")
    print(f"[Models]: Researcher={r['models']['researcher']}, Editor={r['models']['editor']}")


Testing pipeline with multiple questions:


Question 1: What are 3 key benefits of using small language models?

[FINAL]: <|channel|>analysis<|message|>The user wants a rewrite of the entire block of text. The rewrite should be professional, include a one-sentence executive summary first, improve clarity, keep bullet structure if present. The user has provided a large amount of text. The user wants a rewrite of that te...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 2: How does RAG improve AI accuracy?

[FINAL]: <|channel|>final<|message|>**RAG (Retrieval‑Augmented Generation) empowers AI to produce highly accurate, contextually relevant responses by combining a retrieval system with a large language model (LLM).**<|return|>...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 3: Why is local inference important for privacy?

[FINAL]: <|channel|>final<|message|>**Local inference—processing data d


---

**Avertissement** :  
Ce document a été traduit à l'aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d'assurer l'exactitude, veuillez noter que les traductions automatisées peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d'origine doit être considéré comme la source faisant autorité. Pour des informations critiques, il est recommandé de recourir à une traduction humaine professionnelle. Nous déclinons toute responsabilité en cas de malentendus ou d'interprétations erronées résultant de l'utilisation de cette traduction.
